In [ ]:
from quantum_badger import *
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats=['svg'] 

colors = ['#8ECAE6', '#219EBC', '#023047', '#FFCB47', '#FFB703', '#FB8500', '#BB0A21']

# Algorithm Demonstration

<img src="images/qb_image_1.png" alt="Drawing" style="width: 400px;"/>

## Compute all possible probabilities exactly

Here we generate [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) with probabilities of all basis states. 

First of all, you need to initialize the GBS device. We do not recommend run this notebook if the number of clicked detectors of the device, denoted here as `n`, is larger than 30. 

Further we will briefly reproduce the logic of our method, which is described in detail [here](https://arxiv.org/pdf/2106.01445.pdf). 

In [ ]:
#path = create_path(filename='method_details.ipynb')
path = "/Users/anastasiacertkova/Desktop/Coding/data/06_14-22_08_2023"
m = 8
#  Number of input squeezed states
n = round(m/2) 
#  Squeezing parameter of the input squeezed vacuum states
r = 1.6
#  Number of beam splitters 
n_BS = m**2

M, U = choose_default_device(m, r, path=path)

In [ ]:
df_basis = get_basis_df(M)
df_basis.info()
df_basis.head()

#### Find probability of a specific sample in DataFrame

In [ ]:
sample = [0]*m 

# or sample ='00000' without convert_list_to_str() method

df_basis["probability_exact"].loc[convert_list_to_str(sample)] 

#### Total probability mass function of a small Gaussian Boson Sampling device

In [ ]:
pmf_values = [sum(df_basis["probability_exact"][df_basis["n_clicks"] == n]) for n in range(m+1)]

plt.vlines(
    range(m+1), 
    0,
    pmf_values, 
    color = colors[0],
    linestyles='dashed'
)

plt.plot(
    range(m+1), 
    pmf_values,
    'o-',
    color = colors[1]
)

plt.yscale('log')
plt.xlabel("Number of clicks", fontsize=12)
plt.ylabel("Probability mass function", fontsize=12)
plt.title("PMF for Gaussian Boson Sampling", fontsize=17);
plt.show()

## What do sectors look like?
---
#### Total probability mass function of sectors 

In [ ]:
sample = [1]*(m-2) + [0]*2 # just for example
n_clicked = sum(sample)

P_sectors = prob_sectors_exact(M, sample=sample)

for nu in range(n_clicked,n_clicked*10, n_clicked):
    plt.plot(
        range(n_clicked+1),
        [P_sectors[j,nu] for j in range(n_clicked+1)],
        '--' ,
        label = 'k='+str(nu)
)
plt.yscale('log')
plt.ylim(10**(-11), 10**(-1))
plt.legend(prop={'size':10}, loc='lower left')
plt.xlabel("Number of clicks", fontsize=12)
plt.ylabel("Probability mass function", fontsize=12)
plt.title("PMF for sectors", fontsize=17);
plt.show()

---
#### Probability mass function of sectors for a sample

In [ ]:
sample = [1]*(m-2) + [0]*2 # just for example
n_clicked = sum(sample)
nu_max = 10*n_clicked


plt.plot(
        range(nu_max),
        [P_sectors[n_clicked,nu] for nu in range(nu_max)],
        '-' 
)
plt.yscale('log')
#plt.legend(prop={'size':10}, loc='lower left')
plt.xlabel("Sectors", fontsize=12)
plt.ylabel("Probability mass function", fontsize=12)
plt.title(f"PMF of sectors for {n_clicked} clicked detectors", fontsize=17);
plt.show()

---
#### Convergence of a sum over sectors to the exact result 

In [ ]:
# Comparison of calculation of the exact probabilities with and without sectors

sample = [1]*(m-2) + [0]*2 # just for example
n_clicked = sum(sample)
nu_max = 10*n_clicked


P_sectors =  prob_sectors_exact(M, sample=sample)

P_ = 0
for nu in range(nu_max):
    P_ += P_sectors[n_clicked,nu]
    

print("Exact probability: ", "{:.3e}".format(P_)) 
print("Probability summed over sectors: ",
      "{:.3e}".format(df_basis["probability_exact"].loc[convert_list_to_str(sample)]))

#### Approximate probability mass function of sectors for a sample

--- 
#### Comparison with Strawberry Fields Library

In [ ]:
import strawberryfields as sf
from strawberryfields.ops import *
from thewalrus import threshold_detection_prob

def sf_result(r_s, phi_s, ind, phi, psi, eta, n_bs, m):

    """
    Here we check our exact probabilities results 
    with the Walrus https://github.com/XanaduAI/thewalrus
    and Strawberry Fields https://github.com/XanaduAI/strawberryfields libraries.  
    """

    prog = sf.Program(m)

    eng = sf.Engine("gaussian")

    with prog.context as q:

        for i in range(m):

            Sgate(r_s[i], phi_s[i]) | q[i]

        for k in range(n_bs):

            Rgate(phi[k]) | q[ind[k,0]]
            BSgate(eta[k]) | (q[ind[k,1]], q[ind[k,0]])
            Rgate(psi[k]) | q[ind[k,1]]

    state = eng.run(prog).state

    mu = state.means()
    cov = state.cov()

    return mu, cov

In [ ]:
r_, phi_ = import_initial_state(path, "/initial_state.dat")
ind, phi, psi, eta, n_bs, m = import_parameters_interferometer(path, "/parameters_of_interferometer.dat")

mu, cov = sf_result(r_, phi_, ind, phi, psi, eta, n_bs, m)
proba_sf = []

for s in df_basis.index.to_list():
    
    proba_sf.append(
        threshold_detection_prob(mu, cov, convert_str_to_list(s)).real
    )

    
df_basis["probability_exact_sf"] = proba_sf
proba_qb = df_basis["probability_exact"].to_list()

print(f"Fidelity between two distributions: {round(fidelity(proba_qb,proba_sf),4)}")
df_basis.head()

---
### Noisy Interferometer Matrix Generation

In [ ]:
error = 0.05

ind, phi, psi, eta, n_bs, m = import_parameters_interferometer(path, '/parameters_of_interferometer.dat')
U_appr = interferometer_approx(n_BS, ind, phi, psi, eta, error,  m)

export_complex_matrix(path + r"/matrix_U_appr.dat", U_appr)

print("Frobenius distance between U and U_appr:", round(frobenius_distance(U, U_appr),2) )

---
### Tests for probabilities of all states for 2 interferometers 

In [ ]:
df_basis_exact = get_basis_df(M)

M_appr = set_device_parameters(r, A, U_appr)

df_basis_appr = get_basis_df(M_appr)


p = df_basis_exact["probability_exact"].to_list()
q = df_basis_appr["probability_exact"].to_list()

print(f"Relative Weighted Error: {round(relative_weighted_error(p,q),4)}")
print(f"Total Variation Distance: {round(total_variation_distance(p,q),4)}")
print(f"Fidelity: {round(fidelity(p,q),4)}")
print(f"Cosine Similarity:{round(cosine_similarity(p,q),4)}")
print(f"Cross Entropy: {round(cross_entropy(p,q),4)}")

--- 
### Performance of the algorithm

#### The worst case: exact computation of probabilities for all basis states

In [ ]:
# Gives exponential time growth 
import time

time_in_sec = []
n_modes = 15

for n in range (4, n_modes, 2):
    # Generate a random matrix with appropriate size 
    M, U = choose_default_device(n, r, path=None)
    
    start_time = time.perf_counter()

    # Obtain all possible samples for theshold detection
    all_permutations = threshold_basis_set(n)

    # Calculate probabilities for all possible samples 
    probabilities_exact = []

    for s in all_permutations:
        probabilities_exact.append(prob_exact(s, M))
        
    time_in_sec.append(time.perf_counter() - start_time)
    

In [ ]:
plt.loglog([n for n in range (4, n_modes, 2)], time_in_sec)
plt.xlabel("Number of modes")
plt.ylabel("Time")
plt.title("Log-Log Plot: GBS with threshold detectors");

In [ ]:
# All states of threshold detectors computation:  

time_in_sec = []
n_modes = 24

for n in range (4, n_modes, 2):
    
    start_time = time.perf_counter()

    # Obtain all possible samples for theshold detection
    all_permutations = threshold_basis_set(n)

    
    time_in_sec.append(time.perf_counter() - start_time)

In [ ]:
plt.loglog([n for n in range (4, n_modes, 2)], time_in_sec)
plt.xlabel("Number of modes")
plt.ylabel("Time")
plt.title("Log-Log Plot: all states");

In [ ]:
# Just for comparison, we also plot a uniform sampling 
time_in_sec = []
n_modes = 500
batch_size = 10 

for n in range (4, n_modes, 2):
    
    start_time = time.perf_counter()

    samples = uniform_sampling_tr(batch_size, n, 2*n)
    
    time_in_sec.append(time.perf_counter() - start_time)

In [ ]:
plt.loglog([n for n in range (4, n_modes, 2)], time_in_sec)
plt.xlabel("Number of clicked detectors")
plt.ylabel("Time")
plt.title("Log-Log Plot: Uniform Sampling");